In [21]:
!pip install gymnasium[accept-rom-license]

In [22]:
from gymnasium.utils.play import play
import gymnasium as gym


def play_env():
    env = gym.make("ALE/BankHeist-v5", render_mode="rgb_array")
    env.metadata['render_fps'] = 15
    play(env, zoom=3)

play_env()

NamespaceNotFound: Namespace ALE not found. Have you installed the proper package for ALE?

In [3]:
import gymnasium
import torch
import numpy as np
from gymnasium.wrappers import Monitor

# Загрузка модели из файла .pt
model = torch.load('your_model.pt')
model.eval()

# Функция для преобразования изображения в формат, подходящий для модели
def preprocess_observation(obs):
    # Преобразование изображения в формат PyTorch
    return torch.from_numpy(obs).float().unsqueeze(0)

# Функция для выбора действия с помощью модели
def select_action(obs):
    with torch.no_grad():
        # Получение предсказания модели
        output = model(obs)
        # Выбор действия на основе предсказания
        action = torch.argmax(output).item()
    return action

# Функция для игры с использованием модели
def play_game(env_name, model):
    env = gym.make(env_name)
    env = Monitor(env, './videos', force=True) # Запись видео игры
    obs = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Преобразование наблюдения и выбор действия с помощью модели
        obs_tensor = preprocess_observation(obs)
        action = select_action(obs_tensor)

        # Применение действия к среде и получение нового состояния
        obs, reward, done, _ = env.step(action)
        total_reward += reward

    env.close()
    print("Total Reward:", total_reward)

# Запуск игры с использованием модели
play_game('BankHeist-v5', model)